# Geometric distribution example

In [1]:
# Load generated data from file:
import pickle

from evaluation import *

runs = []
num_chains = 10
for i in range(num_chains):
    with open(f"samples_produced/geometric{i}__count1000_eps0.1_leapfrogsteps5.pickle", "rb") as f:
        runs.append(pickle.load(f))

In [2]:
# Read Anglican files:
for i in range(num_chains):
    for method in anglican_methods:
        runs[i][method] = {}
        with open(f"anglican_samples/geo/{method}{i}.txt") as f:
            contents = f.read()
            # Skip parentheses:
            contents = contents[1:-1]
            # Extract numbers:
            numbers = [int(x) for x in contents.split()]
            runs[i][method]["samples"] = numbers

# Read timings:
if anglican_methods:
    timings = parse_anglican_timings("anglican_samples/geo/timing.txt")
    for method in anglican_methods:
        for i in range(len(runs)):
            runs[i][method]["time"] = timings[method][i]

In [3]:
thinned_runs = thin_runs(all_methods, runs)
values = collect_values(all_methods, thinned_runs)
print_running_time(all_methods, runs, thinned_runs)


Running times:
hmc: 405.90s    0.0406s per sample (after thinning)
is: 68.15s    0.0068s per sample (after thinning)
lmh: 3.42s    0.0003s per sample (after thinning)
pgibbs: 1.43s    0.0001s per sample (after thinning)
rmh: 4.87s    0.0005s per sample (after thinning)
ipmcmc: 1.07s    0.0001s per sample (after thinning)


In [4]:
def total_variational_distance(samples, p = 0.2):
    samples = sorted(samples)
    N = len(samples)
    mx = int(samples[-1])
    freq = [0 for _ in range(mx + 1)]
    for sample in samples:
        freq[int(sample)] += 1
    dist = 0
    for x in range(1,mx + 1):
        dist += abs(freq[x] / N - (1 - p)**(x-1)*p)
    dist += (1 - p)**mx # unsampled tail
    return dist / 2

print("Total variation distance from the ground truth:")
for method in all_methods:
    print(f"{method}:  {total_variational_distance(values[method]):.4f}")

Total variation distance from the ground truth:
hmc:  0.0136
is:  0.0165
lmh:  0.0224
pgibbs:  0.0158
rmh:  0.0196
ipmcmc:  0.0164
